## Import thư viện

In [1]:
!pip install -Uqq underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 41.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00


In [2]:
!pip install torchtext==0.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [3]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from itertools import chain
from collections import Counter
from underthesea import word_tokenize
import re
import unicodedata
import string
import matplotlib.pyplot as plt
import seaborn as sns
import underthesea
from bs4 import BeautifulSoup
import logging
import os
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import random

import torchtext.vocab as vocab

# Tăng cường dữ liệu 
Tăng cường dữ liệu có label là 0 vì dữ liệu quá ít so với các label khác

In [6]:

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Từ điển đồng nghĩa mở rộng
synonym_dict = {
    'lỗi': ['hỏng', 'trục trặc', 'không hoạt động'],
    'nghiêm trọng': ['đáng kể', 'lớn', 'khủng khiếp', 'trầm trọng'],
    'nhỏ': ['hạn chế', 'khiêm tốn'],
    # 'mỏi': ['đau', 'mệt mỏi', 'nhức nhối', 'khó chịu'],
    'nhiễu': ['lỗi hình', 'sai lệch', 'méo mó', 'biến dạng'],
    'chậm': ['lỗi thời', 'ùn tắc', 'tệ hại', 'ngắc ngoải', 'hơi chậm', 'trì trệ nhẹ', 'không nhanh lắm'],
    'không': ['mất', 'từ chối'],
    'tắt': ['ngắt', 'đột ngột dừng', 'sập'],
    'thất_vọng': ['tức giận', 'buồn bã'],
    'kém': ['yếu', 'tệ', 'thấp kém'],
    # 'ổn': ['khá tốt', 'đạt yêu cầu'],
    # 'nhanh': ['tốc độ tốt', 'hiệu quả'],
    # 'tốt': ['đẹp', 'hài lòng'],
    # 'hơi': ['tương đối', 'có phần', 'kém chút'],
    'tiếc': ['hơi buồn', 'thất vọng', 'chưa ưng']
}

# Hàm thay thế từ đồng nghĩa
def synonym_replacement(text, prob=0.3):
    if not text:
        return text
    words = text.split()
    new_words = words.copy()
    replaced = False
    for i in range(len(new_words)):
        if random.random() < prob:
            for word in synonym_dict.keys():
                if word in new_words[i]:
                    new_words[i] = random.choice(synonym_dict.get(word, [new_words[i]]))
                    replaced = True
                    break
    if not replaced:
        logging.warning(f"Không có từ nào trong '{text}' được thay thế do không khớp với từ điển đồng nghĩa.")
    return ' '.join(new_words)

# Hàm tăng cường dữ liệu chỉ với synonym_replacement
def augment_data(data, augment_labels=[0], num_augment=2):
    augmented_data = []
    for idx, row in data.iterrows():
        augmented_data.append({'cleaned_data': row['cleaned_data'], 'label': row['label']})
        if row['label'] in augment_labels:
            for _ in range(num_augment):
                aug_text = synonym_replacement(row['cleaned_data'])
                augmented_data.append({'cleaned_data': aug_text, 'label': row['label']})
    return pd.DataFrame(augmented_data)

# Hàm chính
def main(input_file, output_file='augmented_data.xlsx'):
    try:
        data = pd.read_excel(input_file)
        logging.info(f"Đọc thành công dữ liệu với {len(data)} mẫu.")
    except Exception as e:
        logging.error(f"Lỗi khi đọc tệp Excel: {e}")
        return
    
    augmented_data = augment_data(data, augment_labels=[0], num_augment=2)
    logging.info(f"Tổng số mẫu sau khi tăng cường: {len(augmented_data)}")
    
    try:
        augmented_data.to_excel(output_file, index=False)
        logging.info(f"Lưu dữ liệu đã tăng cường vào {output_file}")
    except Exception as e:
        logging.error(f"Lỗi khi lưu tệp: {e}")

if __name__ == "__main__":
    input_file = '/kaggle/input/final-data/final_data.xlsx'
    output_file = 'augmented_data.xlsx'
    main(input_file, output_file)

In [7]:
data = pd.read_excel("/kaggle/working/augmented_data.xlsx")
data.head()

,cleaned_data,label
0,shop giao sản_phẩm sử_dụng không hài_lòng sản_...,0
1,shop giao sản_phẩm sử_dụng không hài_lòng sản_...,0
2,shop giao sản_phẩm sử_dụng không hài_lòng sản_...,0
3,đóng_gói cẩn_thận sp nhỏ gọn hài_lòng cám_ơn tiki,1
4,sản_phẩm nhỏ gọn giao hàng nhanh,2


In [8]:
data['label'].value_counts()

label
2    14907
0    12222
1     7925
Name: count, dtype: int64

# Load pretrained Vietnamese word2vec
# Embedding 
# Huấn luyện với mô hình lstm

In [9]:
# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 1. Tạo file vi_word2vec.txt từ word2vec_vi_words_100dims.txt
print("Tạo tệp vi_word2vec.txt...")
with open("/kaggle/input/word2vec/word2vec_vi_words_100dims.txt", "r", encoding='utf-8') as f:
    first_line = f.readline()
    num_words, dim = [int(word) for word in first_line.split()]
    with open("vi_word2vec.txt", "w", encoding='utf-8') as f_out:
        for _ in tqdm(range(num_words)):
            f_out.write(f.readline())

# 2. Tải Word2Vec
word_embedding = vocab.Vectors(name="vi_word2vec.txt", unk_init=torch.Tensor.normal_)

# 3. Xây dựng Vocabulary
class Vocabulary:
    def __init__(self, min_freq=3):
        self.word2id = dict()
        self.word2id['<pad>'] = 0
        self.word2id['<unk>'] = 1
        self.unk_id = self.word2id['<unk>']
        self.id2word = {v: k for k, v in self.word2id.items()}
        self.min_freq = min_freq

    def __getitem__(self, word):
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        return word in self.word2id

    def __len__(self):
        return len(self.word2id)

    def add(self, word):
        if word not in self:
            word_index = self.word2id[word] = len(self.word2id)
            self.id2word[word_index] = word
            return word_index
        return self[word]

    @staticmethod
    def tokenize_corpus(corpus):
        print("Tokenize the corpus...")
        tokenized_corpus = list()
        for document in tqdm(corpus):
            if isinstance(document, str) and document.strip():
                tokenized_document = word_tokenize(document)
                tokenized_corpus.append(tokenized_document)
            else:
                tokenized_corpus.append([])
        return tokenized_corpus

    def build_vocab(self, tokenized_corpus):
        print("Building vocabulary...")
        word_counts = {}
        for document in tokenized_corpus:
            for word in document:
                word_counts[word] = word_counts.get(word, 0) + 1
        
        for word, freq in word_counts.items():
            if freq >= self.min_freq:
                self.add(word)

    def corpus_to_tensor(self, corpus, is_tokenized=False):
        if is_tokenized:
            tokenized_corpus = corpus
        else:
            tokenized_corpus = self.tokenize_corpus(corpus)
        indicies_corpus = list()
        for document in tqdm(tokenized_corpus):
            indicies_document = torch.tensor(list(map(lambda word: self[word], document)),
                                             dtype=torch.int64)
            indicies_corpus.append(indicies_document)
        return indicies_corpus

# 4. Tạo Dataset
class SentimentDataset(Dataset):
    def __init__(self, vocab, data, text_column='cleaned_data', label_column='label'):
        self.vocab = vocab
        self.pad_idx = vocab["<pad>"]

        self.reviews_list = list(data[text_column].fillna(''))
        self.labels_list = list(data[label_column])

        self.tokenized_reviews = self.vocab.tokenize_corpus(self.reviews_list)
        self.vocab.build_vocab(self.tokenized_reviews)
        self.tensor_data = self.vocab.corpus_to_tensor(self.tokenized_reviews, is_tokenized=True)
        self.tensor_label = torch.tensor(self.labels_list, dtype=torch.long)

    def __len__(self):
        return len(self.tensor_data)

    def __getitem__(self, idx):
        return self.tensor_data[idx], self.tensor_label[idx]

    def collate_fn(self, examples):
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
        reviews = [e[0] for e in examples]
        reviews = torch.nn.utils.rnn.pad_sequence(reviews, batch_first=False, padding_value=self.pad_idx)
        reviews_lengths = torch.tensor([len(e[0]) for e in examples])
        labels = torch.tensor([e[1] for e in examples])
        return {"reviews": (reviews, reviews_lengths), "labels": labels}

# 5. Tải và chuẩn bị dữ liệu
print("Đang tải dữ liệu...")
data = pd.read_excel("/kaggle/working/augmented_data.xlsx")

# Kiểm tra phân bố nhãn
label_counts = data['label'].value_counts()
logging.info(f"Phân bố nhãn: {label_counts.to_dict()}")

# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])

# 6. Tạo vocabulary và dataset
vocab = Vocabulary(min_freq=3)
train_dataset = SentimentDataset(vocab, train_data)
test_dataset = SentimentDataset(vocab, test_data)

# 7. Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=train_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=test_dataset.collate_fn)

print(f"Số mẫu huấn luyện: {len(train_dataset)}")
print(f"Số mẫu kiểm tra: {len(test_dataset)}")
print(f"Kích thước từ điển: {len(vocab)}")
print("\n=== Xem dữ liệu trong train_loader (sau padding) ===")
for batch in train_loader:
    reviews, reviews_lengths = batch["reviews"]
    labels = batch["labels"]
    print("Batch reviews (padded tensor):")
    print(reviews)
    print("Độ dài thực tế của mỗi mẫu trong batch:", reviews_lengths)
    print("Nhãn của batch:", labels)
    # Chuyển tensor padded thành văn bản
    for i in range(reviews.shape[1]):  # Duyệt qua từng mẫu trong batch
        words = [vocab.id2word[idx.item()] for idx in reviews[:, i] if idx.item() != vocab["<pad>"]]
        print(f"  Mẫu {i} trong batch: {' '.join(words)}")
    break  # Chỉ hiển thị batch đầu tiên

# 5. Định nghĩa mô hình LSTM
class SentimentLSTM(nn.Module):
    def __init__(self, vocab, pretrained_embedding, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        self.vocab = vocab
        vocab_size = len(vocab)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # Ánh xạ vector từ pretrained_embedding cho các từ trong vocab
        self.init_embeddings(pretrained_embedding)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=False)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def init_embeddings(self, pretrained_embedding):
        embedding_dim = pretrained_embedding.vectors.size(1)
        vocab_size = len(self.vocab)
        # Khởi tạo ngẫu nhiên cho tất cả các từ trong vocab
        self.embedding.weight.data.normal_(mean=0, std=0.1)
        # Gán vector từ pretrained_embedding cho các từ có trong vocab
        for word, idx in self.vocab.word2id.items():
            if word in pretrained_embedding.stoi:
                self.embedding.weight.data[idx] = pretrained_embedding.vectors[pretrained_embedding.stoi[word]]
        # Đảm bảo vector của <pad> là 0
        self.embedding.weight.data[self.vocab["<pad>"]] = torch.zeros(embedding_dim)
        # Không huấn luyện embedding
        self.embedding.weight.requires_grad = False

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden[-1,:,:]
        return self.fc(self.dropout(hidden))

# 6. Hàm huấn luyện và đánh giá
def train_epoch(model, data_loader, criterion, optimizer, clip_value=1.0):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    batch_count = 0

    for batch in tqdm(data_loader, desc="Training"):
        reviews, reviews_lengths = batch["reviews"]
        labels = batch["labels"]

        optimizer.zero_grad()
        predictions = model(reviews, reviews_lengths)
        loss = criterion(predictions, labels)

        # Kiểm tra loss có hợp lệ không
        if torch.isnan(loss) or torch.isinf(loss):
            logging.warning(f"Loss is invalid at batch {batch_count}: {loss.item()}")
            continue

        loss.backward()

        # Gradient clipping để tránh gradient explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)

        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        epoch_acc += (predicted == labels).sum().item()
        batch_count += 1

        # In thông tin từng batch (tuỳ chọn)
        if batch_count % 10 == 0:
            logging.info(f"Batch {batch_count}/{len(data_loader)} - Loss: {loss.item():.4f}, Acc: {(predicted == labels).sum().item() / labels.size(0):.4f}")

    if batch_count == 0:
        logging.warning("No valid batches to train on!")
        return 0, 0

    return epoch_loss / batch_count, epoch_acc / total

def evaluate(model, data_loader, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            reviews, reviews_lengths = batch["reviews"]
            labels = batch["labels"]

            predictions = model(reviews, reviews_lengths)
            loss = criterion(predictions, labels)

            epoch_loss += loss.item()
            _, predicted = torch.max(predictions, 1)
            total += labels.size(0)
            epoch_acc += (predicted == labels).sum().item()

            all_preds.extend(predicted.numpy())
            all_labels.extend(labels.numpy())

    return epoch_loss / len(data_loader), epoch_acc / total, all_preds, all_labels

# Thêm class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.array([0, 1, 2]), y=train_data['label'])

class_weights = torch.tensor([1.0, 2.0, 0.5], dtype=torch.float)
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha if alpha is not None else torch.ones(3)
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha[targets] * (1 - pt) ** self.gamma * ce_loss)
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

# Sử dụng Focal Loss thay vì CrossEntropyLoss
criterion = FocalLoss(gamma=2.0, alpha=torch.tensor([1.0, 2.0, 0.5]))
# criterion = nn.CrossEntropyLoss(weight=class_weights)

# Khởi tạo mô hình với hyperparameters mới
model = SentimentLSTM(
    vocab=vocab,
    pretrained_embedding=word_embedding,
    embedding_dim=100,
    hidden_dim=512,
    output_dim=3,
    n_layers=3,
    bidirectional=True,
    dropout=0.3
)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Huấn luyện với early stopping
best_test_loss = float('inf')
patience = 5
counter = 0
n_epochs = 20

print("\nBắt đầu huấn luyện...")
for epoch in range(n_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, clip_value=1.0)
    test_loss, test_acc, test_preds, test_labels = evaluate(model, test_loader, criterion)
    scheduler.step()

    print(f'Epoch {epoch+1}/{n_epochs}:')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

    # Early stopping
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        counter = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered!")
            break

# Đánh giá
print("\nBáo cáo phân loại trên tập kiểm tra:")
print(classification_report(test_labels, test_preds, target_names=[str(i) for i in range(3)]))

Tạo tệp vi_word2vec.txt...


100%|█████████▉| 1587506/1587507 [01:10<00:00, 22647.07it/s]


Đang tải dữ liệu...
Tokenize the corpus...


100%|██████████| 28043/28043 [00:23<00:00, 1213.40it/s]


Building vocabulary...


...

Epoch 14/20:
  Train Loss: 0.1413, Train Acc: 0.7535
  Test Loss: 0.1934, Test Acc: 0.7413


Evaluating: 100%|██████████| 220/220 [00:46<00:00,  4.78it/s]


Epoch 15/20:
  Train Loss: 0.1380, Train Acc: 0.7603
  Test Loss: 0.2106, Test Acc: 0.7364


Evaluating: 100%|██████████| 220/220 [00:44<00:00,  4.94it/s]

Epoch 16/20:
  Train Loss: 0.1250, Train Acc: 0.7745
  Test Loss: 0.2052, Test Acc: 0.7401
Early stopping triggered!

Báo cáo phân loại trên tập kiểm tra:
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      2444
           1       0.47      0.67      0.55      1585
           2       0.88      0.63      0.73      2982

    accuracy                           0.74      7011
   macro avg       0.74      0.74      0.73      7011
weighted avg       0.78      0.74      0.75      7011

